In [ ]:
gist = "https://gist.github.com/a8037ba6f9df4d921f8c271682c2d4e0.git"
sha1 = "7fff59d1ab70e676c65ff5362559e6ee42c40f00"

import LibGit2
using Pkg
env = joinpath(@__DIR__, ".env", "ConcolicFuzzerTalk")
isdir(env) || LibGit2.clone(gist, env)
repo = LibGit2.GitRepo(env)
LibGit2.fetch(repo)
LibGit2.checkout!(repo, sha1, force = true)
Pkg.activate(env)
Pkg.instantiate()

In [ ]:
using ConcolicFuzzer

- Concolic Execution based on Cassette
- Uses Z3 as an SMT solver
- Proof properties of integers
- Fuzzing!!!

## General Idea

In [ ]:
function vault(passwd)
    if passwd == 1337
        return "Secret!"
    else
        return "Access denied"
    end
end

## Attacking through fuzzing
- Linear search
- Random inputs
- Concolic execution

In [ ]:
result, symbolic, trace, record = execute(vault, 1)
result, symbolic

Simplified trace:
```
vault(arg_1)
  Cond ⇐ ==(arg_1, 1337)
    Cond ⇐ ===(arg_1, 1337)
  false ⇐ assert(Cond)
```

In [ ]:
stream = ConcolicFuzzer.filter(trace)

### Turning it into a SMT program

In [ ]:
z3model = ConcolicFuzzer.symbolic(stream);

```smt
(declare-const arg (_ BitVec 64))
(declare-const cond Bool)
(assert (= cond (= arg (_ bv1337 64))))
(assert (= cond false))
```

This is what `z3model = ConcolicFuzzer.symbolic(stream)` produces.

In [ ]:
sat, model = ConcolicFuzzer.runZ3(copy(z3model))
if sat
    inputs = ConcolicFuzzer.parseZ3(model)
    print("model is satisfiable with inputs ", inputs[1])
end

### Open sesame
1. Flip the condition
2. Run Z3
3. Profit

In [ ]:
val = stream[2]
stream[2] = (val[1], Some(true), val[3])
stream

In [ ]:
z3model = ConcolicFuzzer.symbolic(stream);

In [ ]:
sat, model = ConcolicFuzzer.runZ3(copy(z3model))
if sat
    inputs = ConcolicFuzzer.parseZ3(model)
    print("model is satisfiable with inputs ", inputs[1])
end

## Proofs

We can also explicitly prove invariants of our program

In [ ]:
import ConcolicFuzzer: assert, prove

In [ ]:
function proof_me(x)
    assert(x<10)
    y = x - 10
    prove(y < 0)
    return y
end

In [ ]:
sat, (inputs, hidden_inputs, assignments) = check(proof_me, 1)
sat, inputs

In [ ]:
proof_me(-9223372036854775808)

## Fuzzing
The process of exploring a program.

Goal:
- Obtain a set of inputs that will explore all branches of a function

In [ ]:
function pure_madness(x::T) where T
    if x < typemax(T)
        return 42
    end
    return 9001
end

In [ ]:
pure_madness(rand(Int))

In [ ]:
tested, errored = fuzz(pure_madness, Int64);
map(c-> c[2] => c[1], tested)

In [ ]:
typemax(Int64)

In [ ]:
tested, errored = fuzz(pure_madness, Union{Int64, UInt64});
map(c-> c[2] => c[1], tested)

In [ ]:
tested, errored = fuzz(pure_madness, Integer);

In [ ]:
# We are missing one of the Bool cases
map(c-> c[2] => c[1], tested)

## More complicated features
- `rand`
- `loops`

In [ ]:
function r3()
    x = rand(Int)
    y = rand(Int)

    if x + y < 10
        return 42
    end
    return 12
end

In [ ]:
tested, errored = fuzz(r3);

In [ ]:
map(c-> c[2] => c[1], tested)

In [ ]:
function out_of_bounds(N)
    i = 1
    while i <= N
        @assert i <= 10
        i +=  1
    end
    return i
end

In [ ]:
tested, errored = fuzz_wargs(out_of_bounds, 0);

In [ ]:
map(c-> c[2] => c[1], tested)

### Supports structs 
```julia
struct A
    x::Int
end
function propagate(x)
    a = A(x)
    return a.x
end
```

### Supports Arrays
```julia
function store_and_read(x, i, j)
    A = Array{Int}(undef, 10)
    A[i] = x
    x = 0 # Destroy symbol character of x
    return A[j]
end
```

# Fuzz and check

In [ ]:
function fc(y)
    @assert(0 < y)
    if y < 10
        x = y - 5
    else
        x = y - 7
    end
    ConcolicFuzzer.prove(x < y)
end
result = ConcolicFuzzer.fuzz_and_check(fc, Int)

## Work left to do!
- BigInt support to properly model the Integer domain
- Anything but `<:Integers`
  - String
  - arbitrary primitive types
  - Floating Point
- Handling general `ccall`'s as point of introducing *taint*


## How does it work?

- Taint inputs with Cassette tags
- Trace computations involving tainted values
- Record branches by inserting `assert`
- Perform DFS, by cutting and negating branches
- Build a SMT-2 program
- Solve the constraints to get inputs to explore new branch.
- Iterate